In [ ]:
import asyncio
import nest_asyncio
import websockets
import json

ip = "192.168.7.2"
port=5555
data_add = "gui_data"
control_add = "gui_control"

ws_control_add = f"ws://{ip}:{port}/{control_add}"
ws_data_add = f"ws://{ip}:{port}/{data_add}"

In [ ]:
nest_asyncio.apply() # event loop needs to be nested - otherwise it conflicts with jupyter's event loop

async def send_msg_callback(ws_address,msg):
    async with websockets.connect(ws_address) as ws:
        await ws.send(json.dumps(msg))
        print(f"Sent: {msg}")

async def rec_msg_callback(ws_address):
    async with websockets.connect(ws_address) as ws:
        while True:
            msg = await ws.recv()
            print(f"Received:{msg}") # TODO filter according to msg type

async def start_listener_callback(ws_address):
    await rec_msg_callback(ws_address)

def send_msg(ws_address,msg):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(send_msg_callback(ws_address,msg))

def start_listener(ws_address):
    loop = asyncio.get_event_loop()
    loop.create_task(start_listener_callback(ws_address)) # create_task() is needed so that the listener runs in the background and prints messages as received without blocking the cell

### connect to control ws

In [ ]:
start_listener(ws_control_add) # this gets a "connection" json response each time, so run only once or TODO filter it out


In [ ]:
send_msg(ws_control_add,  {"watcher":[{"cmd":"hi"}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "unwatch", "watchers":['myvar2']}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd":"list"}]})

### connect to data websockets  

In [ ]:
start_listener(ws_data_add)

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "watch", "watchers":['myvar2']}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "unwatch", "watchers":['myvar2']}]})

### watcher class

In [ ]:
import asyncio
import nest_asyncio
import websockets
import json

nest_asyncio.apply()


class Watcher:

    def __init__(self, ip="192.168.7.2", port=5555, data_add="gui_data", control_add="gui_control"):
        """_summary_

        Args:
            ip (str, optional): Remote address IP. Defaults to "192.168.7.2".
            port (int, optional): Remote address port. Defaults to 5555.
            data_add (str, optional): Data endpoint. Defaults to "gui_data".
            control_add (str, optional): Control endpoint. Defaults to "gui_control".
        """
        self.ip = ip
        self.port = port
        self.data_add = data_add
        self.control_add = control_add
        self.ws_control_add = f"ws://{self.ip}:{self.port}/{self.control_add}"
        self.ws_data_add = f"ws://{self.ip}:{self.port}/{self.data_add}"
        
        self.websocket_control = None
        self.websocket_data = None
        
        self.__ctrl_listener = None
        self.__data_listener = None
                
        nest_asyncio.apply() # event loop needs to be nested - otherwise it conflicts with jupyter's event loop

    async def __send_msg_callback(self, ws_address, msg):
        
        if ws_address == self.ws_control_add: # TODO refactor this
            ws = self.websocket_control
        
        elif ws_address == self.ws_data_add:
            ws = self.websocket_data
            
        
        try:
            async with websockets.connect(ws_address) as ws: # here you can use the same websocket for multiple messages -- but avoid using the same for sending and receiving
                await ws.send(json.dumps(msg))
                print(f"Sent: {msg}")
        except Exception as e:
            print(f"Error while sending message: {e}")      
        
    
    async def __rec_msg_callback(self, ws_address):
        
        if ws_address == self.ws_control_add:   # TODO refactor this
            ws = self.websocket_control

        elif ws_address == self.ws_data_add:
            ws = self.websocket_data
            
        try:
            async with websockets.connect(ws_address) as ws:
                while True:
                    msg = await ws.recv()
                    print(f"{self.__hash__} Received:{msg}") # TODO filter according to msg type
        except Exception as e:
            print(f"Error while receiving message: {e}")

    async def __start_listener_callback(self, ws_address):
        await self.__rec_msg_callback(ws_address)

    def __send_msg(self, ws_address, msg):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.__send_msg_callback(ws_address, msg))

    def __start_listener(self, ws_address):
        loop = asyncio.get_event_loop()
        listener_task = loop.create_task(self.__start_listener_callback(ws_address)) # create_task() is needed so that the listener runs in the background and prints messages as received without blocking the cell
        return listener_task
    
    def send_ctrl_msg(self, msg):
        self.__send_msg(self.ws_control_add, msg)
    
    def start_ctrl_listener(self):
        self.__ctrl_listener = self.__start_listener(self.ws_control_add)
    
    def start_data_listener(self):
        self.__data_listener = self.__start_listener(self.ws_data_add)
        
    def start(self):
        if self.__ctrl_listener is None: # avoid duplicate listeners
            self.start_ctrl_listener()
        if self.__data_listener is None:
            self.start_data_listener()
    
    def stop(self):  
        if self.__ctrl_listener is not None:
            self.__ctrl_listener.cancel()
            self.__ctrl_listener = None # empty the listener 
        if self.__data_listener is not None:
            self.__data_listener.cancel()
            self.__data_listener = None # empty the listener


In [6]:
watcher = Watcher()
watcher2 = Watcher()


In [ ]:
watcher.send_ctrl_msg({"watcher":[{"cmd": "watch", "watchers":['myvar2']}]})

In [7]:
watcher.start()  

<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:{"event":"connection","projectName":"watcher"}
<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'1/f'
<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'\x00\x00i?\x00\x00i?\x00\x00i?\x00 i?\x00 i?\x00 i?\x00`i?\x00`i?\x00`i?\x00`i?\x00`i?\x00\x80i?\x00\x80i?\x00\x80i?\x00\xc0i?\x00\xc0i?\x00\xc0i?\x00\xc0i?\x00\xc0i?\x00\xe0i?\x00\xe0i?\x00\xe0i?\x00 j?\x00 j?\x00 j?\x00 j?\x00 j?\x00@j?\x00@j?\x00@j?\x00\x80j?\x00\x80j?\x00\x80j?\x00\x80j?\x00\x80j?\x00\xa0j?\x00\xa0j?\x00\xa0j?\x00\xe0j?\x00\xe0j?\x00\xe0j?\x00\xe0j?\x00\xe0j?\x00\x00k?\x00\x00k?\x00\x00k?\x00@k?\x00@k?\x00@k?\x00@k?\x00@k?\x00`k?\x00`k?\x00`k?\x00\xa0k?\x00\xa0k?\x00\xa0k?\x00\xa0k?\x00\xa0k?\x00\xc0k?\x00\xc0k?\x00\xc0k?\x00\x00l?\x00\x00l?\x00\x00l?\x00\x00l?\x00\x00l?\x00 l?\x00 l?\x00 l?\x00@l?\x00@l?\x00@l?\x00@l?\x00@l?\x00\x80l?\x00\x80l?\x00\x80l?\x00\xa0l?\x00\xa0l?\x00\xa0l?\x00\xa0l?\

In [ ]:

watcher.stop()

In [8]:
watcher2.start()
watcher.start()# calling watcher again does not duplicate prints 

<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'1/f'
<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'\x00\xe0a?\x00\xe0a?\x00\xe0a?\x00\x00b?\x00\x00b?\x00\x00b?\x00@b?\x00@b?\x00@b?\x00@b?\x00@b?\x00`b?\x00`b?\x00`b?\x00\xa0b?\x00\xa0b?\x00\xa0b?\x00\xa0b?\x00\xa0b?\x00\xc0b?\x00\xc0b?\x00\xc0b?\x00\x00c?\x00\x00c?\x00\x00c?\x00\x00c?\x00\x00c?\x00 c?\x00 c?\x00 c?\x00`c?\x00`c?\x00`c?\x00`c?\x00`c?\x00\x80c?\x00\x80c?\x00\x80c?\x00\xc0c?\x00\xc0c?\x00\xc0c?\x00\xc0c?\x00\xc0c?\x00\xe0c?\x00\xe0c?\x00\xe0c?\x00 d?\x00 d?\x00 d?\x00 d?\x00 d?\x00@d?\x00@d?\x00@d?\x00\x80d?\x00\x80d?\x00\x80d?\x00\x80d?\x00\x80d?\x00\xa0d?\x00\xa0d?\x00\xa0d?\x00\xc0d?\x00\xc0d?\x00\xc0d?\x00\xc0d?\x00\xc0d?\x00\x00e?\x00\x00e?\x00\x00e?\x00 e?\x00 e?\x00 e?\x00 e?\x00 e?\x00`e?\x00`e?\x00`e?\x00\x80e?\x00\x80e?\x00\x80e?\x00\x80e?\x00\x80e?\x00\xc0e?\x00\xc0e?\x00\xc0e?\x00\xe0e?\x00\xe0e?\x00\xe0e?\x00\xe0e?\x00\xe0e?\x00 f?\x00 f?\x00 f?\x00@f?\x0

<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:{"event":"connection","projectName":"watcher"}
<method-wrapper '__hash__' of Watcher object at 0x7fca3838bfd0> Received:{"event":"connection","projectName":"watcher"}
<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'1/f'
<method-wrapper '__hash__' of Watcher object at 0x7fca3838bfd0> Received:b'1/f'
<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'\x00\x00\x8b=\x00\x00\x8b=\x00\x00\x8b=\x00\x00\x8d=\x00\x00\x8d=\x00\x00\x8d=\x00\x00\x8e=\x00\x00\x8e=\x00\x00\x8e=\x00\x00\x8e=\x00\x00\x8e=\x00\x00\x90=\x00\x00\x90=\x00\x00\x90=\x00\x00\x91=\x00\x00\x91=\x00\x00\x91=\x00\x00\x91=\x00\x00\x91=\x00\x00\x93=\x00\x00\x93=\x00\x00\x93=\x00\x00\x94=\x00\x00\x94=\x00\x00\x94=\x00\x00\x94=\x00\x00\x94=\x00\x00\x96=\x00\x00\x96=\x00\x00\x96=\x00\x00\x97=\x00\x00\x97=\x00\x00\x97=\x00\x00\x97=\x00\x00\x97=\x00\x00\x99=\x00\x00\x99=\x00\x00\x99=\x00\x00\x9a=\x00\x00\x9a=\x00\

In [9]:
watcher.stop()
watcher2.stop()

<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'1/f'
<method-wrapper '__hash__' of Watcher object at 0x7fca3838bfd0> Received:b'1/f'
<method-wrapper '__hash__' of Watcher object at 0x7fca3838ae30> Received:b'\x00\x00$=\x00\x00$=\x00\x00$=\x00\x00(=\x00\x00(=\x00\x00(=\x00\x00*=\x00\x00*=\x00\x00*=\x00\x00*=\x00\x00*=\x00\x00.=\x00\x00.=\x00\x00.=\x00\x000=\x00\x000=\x00\x000=\x00\x000=\x00\x000=\x00\x004=\x00\x004=\x00\x004=\x00\x006=\x00\x006=\x00\x006=\x00\x006=\x00\x006=\x00\x00:=\x00\x00:=\x00\x00:=\x00\x00<=\x00\x00<=\x00\x00<=\x00\x00<=\x00\x00<=\x00\x00>=\x00\x00>=\x00\x00>=\x00\x00B=\x00\x00B=\x00\x00B=\x00\x00B=\x00\x00B=\x00\x00D=\x00\x00D=\x00\x00D=\x00\x00H=\x00\x00H=\x00\x00H=\x00\x00H=\x00\x00H=\x00\x00J=\x00\x00J=\x00\x00J=\x00\x00N=\x00\x00N=\x00\x00N=\x00\x00N=\x00\x00N=\x00\x00P=\x00\x00P=\x00\x00P=\x00\x00T=\x00\x00T=\x00\x00T=\x00\x00T=\x00\x00T=\x00\x00V=\x00\x00V=\x00\x00V=\x00\x00Z=\x00\x00Z=\x00\x00Z=\x00\x00Z=\x00\x00Z=\x00\x00\\=\x00